## Model: Responsible Business Simulator (RBS)
In this notebook the core (calculation) steps of the RBS model are illustrated.

### (I): Read data
Firstly, we are going to read in the data. To do this we use the **import_case()** function that is part of the **core**-folder. This function takes two arguments: a _file_format_ (csv, json or xlsx) and a _path_ to the folder containing the data-folders. Note that with folder there should a folder with a name that matches the _file_format_.  

In [1]:
from pathlib import Path
from core.import_case import import_case

file_format = 'xlsx'
path = Path.cwd() / 'data' / 'beerwiser'
case_data = import_case(file_format, path)

Let's see how our formatted data looks now. _case_data_ is a dictionary containing arrays for each variable.   

In [2]:
for entry in case_data:
    print(entry,":\n",case_data[entry])

configurations :
 ['use_theme_weights']
configuration_value :
 [0]
key_outputs :
 ['Accidents reduction' 'Water use reduction' 'Production cost reduction']
key_output_unit :
 ['#/year' 'hl/year' '%']
key_output_theme :
 ['People' 'Planet' 'Profit']
themes :
 ['People' 'Planet' 'Profit']
key_output_minimum :
 [nan nan nan]
key_output_maximum :
 [nan nan nan]
key_output_monetary :
 [0 0 0]
key_output_smaller_the_better :
 [0 0 0]
key_output_linear :
 [1 1 0]
key_output_automatic :
 [1 1 1]
key_output_start :
 [nan nan nan]
key_output_end :
 [nan nan nan]
key_output_threshold :
 [nan nan nan]
decision_makers_options :
 ['Focus on water recycling' 'Focus on training' 'Equal spread']
internal_variable_inputs :
 ['Invest in training of employees' 'Invest in water recycling']
decision_makers_option_value :
 [[5.0e+004 2.5e+005]
 [2.0e-323 5.0e+004]
 [9.9e-324 1.5e+005]]
scenarios :
 ['Base case' 'Optimistic' 'Pessimistic']
external_variable_inputs :
 ['Cost of accident' 'Effectiveness water r